In [1]:
!pip install xmltodict

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import warnings
warnings.filterwarnings('ignore')

In [3]:
from urllib.request import urlopen
from urllib.request import Request 
from urllib.parse import urlencode, quote_plus

import urllib.request as ul
import xmltodict
import json
import sys
import io

In [4]:
import requests

In [5]:
from datetime import datetime
start = pd.to_datetime('20130101')
end=pd.to_datetime('20181231')
df=pd.DataFrame(columns=['연월일'])
df['연월일']=pd.date_range(start, end,freq='D')

In [6]:
df

,연월일
0,2013-01-01
1,2013-01-02
2,2013-01-03
3,2013-01-04
4,2013-01-05
...,...
2186,2018-12-27
2187,2018-12-28
2188,2018-12-29
2189,2018-12-30


In [7]:
def make_locdate(x):
    Y, M, D = x.split('-')
    return Y+M+D

In [8]:
df['연월일']=df['연월일'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2191 entries, 0 to 2190
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   연월일     2191 non-null   object
dtypes: object(1)
memory usage: 17.2+ KB


In [9]:
df['locdate'] = df['연월일'].apply(lambda x:make_locdate(x))
df

,연월일,locdate
0,2013-01-01,20130101
1,2013-01-02,20130102
2,2013-01-03,20130103
3,2013-01-04,20130104
4,2013-01-05,20130105
...,...,...
2186,2018-12-27,20181227
2187,2018-12-28,20181228
2188,2018-12-29,20181229
2189,2018-12-30,20181230


In [10]:
url = 'http://apis.data.go.kr/B090041/openapi/service/RiseSetInfoService/getAreaRiseSetInfo'

for i in range(0, df.shape[0]):
    locdate = df.loc[i,'locdate']
    params ={'serviceKey' : '0GE5dffPQ2w31OZOACOdSgFRQzPihrAaslHTL1WR8UajH2KJivTGQfGkmw8pCFaWIr9sG2Vh4YEuaRk4rvCgSg==', 'locdate' : locdate, 'location' : '서울' }
    response = requests.get(url, params)
    resp = xmltodict.parse(response.text)
    #print(resp)
    df.loc[i, 'sunrise'] = resp['response']['body']['items']['item']['sunrise']
    df.loc[i, 'sunset'] = resp['response']['body']['items']['item']['sunset']
    print(i)
print('end')

OrderedDict([('response', OrderedDict([('header', OrderedDict([('resultCode', '00'), ('resultMsg', 'NORMAL SERVICE.')])), ('body', OrderedDict([('items', OrderedDict([('item', OrderedDict([('aste', '1858'), ('astm', '0613'), ('civile', '1754'), ('civilm', '0718'), ('latitude', '3733'), ('latitudeNum', '37.5500000'), ('location', '서울'), ('locdate', '20130101'), ('longitude', '12658'), ('longitudeNum', '126.9666660'), ('moonrise', '2114'), ('moonset', '0946'), ('moontransit', '030358'), ('naute', '1826'), ('nautm', '0645'), ('sunrise', '0747'), ('sunset', '1724'), ('suntransit', '123534')]))])), ('numOfRows', '10'), ('pageNo', '1'), ('totalCount', '1')]))]))])
0
OrderedDict([('response', OrderedDict([('header', OrderedDict([('resultCode', '00'), ('resultMsg', 'NORMAL SERVICE.')])), ('body', OrderedDict([('items', OrderedDict([('item', OrderedDict([('aste', '1859'), ('astm', '0613'), ('civile', '1754'), ('civilm', '0718'), ('latitude', '3733'), ('latitudeNum', '37.5500000'), ('location', 

In [11]:
df

,연월일,locdate,sunrise,sunset
0,2013-01-01,20130101,0747,1724
1,2013-01-02,20130102,0747,1725
2,2013-01-03,20130103,0747,1726
3,2013-01-04,20130104,0747,1727
4,2013-01-05,20130105,0747,1728
...,...,...,...,...
2186,2018-12-27,20181227,0746,1720
2187,2018-12-28,20181228,0746,1721
2188,2018-12-29,20181229,0746,1722
2189,2018-12-30,20181230,0746,1722


In [12]:
df['sunset'] = df['sunset'].astype(int)//100
df['sunrise']= df['sunrise'].astype(int)//100

In [13]:
df['일사']= df["sunset"] - df["sunrise"]

In [14]:
df.rename(columns={'연월일':'date'}, inplace=True)

In [15]:
df2 = df[['date','sunrise','sunset','일사']]
df2

,date,sunrise,sunset,일사
0,2013-01-01,7,17,10
1,2013-01-02,7,17,10
2,2013-01-03,7,17,10
3,2013-01-04,7,17,10
4,2013-01-05,7,17,10
...,...,...,...,...
2186,2018-12-27,7,17,10
2187,2018-12-28,7,17,10
2188,2018-12-29,7,17,10
2189,2018-12-30,7,17,10


In [16]:
df2.to_csv('./make_data/0C_train_일출일몰.csv', index=False)

In [19]:
df2['date'] = pd.to_datetime(df2['date'])
df2['year'] = df2['date'].dt.year
df2['month'] = df2['date'].dt.month
df2['day'] = df2['date'].dt.day
df2

,date,sunrise,sunset,일사,year,month,day
0,2013-01-01,7,17,10,2013,1,1
1,2013-01-02,7,17,10,2013,1,2
2,2013-01-03,7,17,10,2013,1,3
3,2013-01-04,7,17,10,2013,1,4
4,2013-01-05,7,17,10,2013,1,5
...,...,...,...,...,...,...,...
2186,2018-12-27,7,17,10,2018,12,27
2187,2018-12-28,7,17,10,2018,12,28
2188,2018-12-29,7,17,10,2018,12,29
2189,2018-12-30,7,17,10,2018,12,30


In [20]:
# test용 2013~2018 평균 구하기
df3= df2[df2['year']>=2013].groupby(['month','day']).mean() 

In [21]:
df3=df3.reset_index()
df3

,month,day,sunrise,sunset,일사,year
0,1,1,7.0,17.0,10.0,2015.5
1,1,2,7.0,17.0,10.0,2015.5
2,1,3,7.0,17.0,10.0,2015.5
3,1,4,7.0,17.0,10.0,2015.5
4,1,5,7.0,17.0,10.0,2015.5
...,...,...,...,...,...,...
361,12,27,7.0,17.0,10.0,2015.5
362,12,28,7.0,17.0,10.0,2015.5
363,12,29,7.0,17.0,10.0,2015.5
364,12,30,7.0,17.0,10.0,2015.5


In [22]:
df3 = df3.iloc[:91]
df3

,month,day,sunrise,sunset,일사,year
0,1,1,7.0,17.0,10.0,2015.5
1,1,2,7.0,17.0,10.0,2015.5
2,1,3,7.0,17.0,10.0,2015.5
3,1,4,7.0,17.0,10.0,2015.5
4,1,5,7.0,17.0,10.0,2015.5
...,...,...,...,...,...,...
86,3,27,6.0,18.0,12.0,2015.5
87,3,28,6.0,18.0,12.0,2015.5
88,3,29,6.0,18.0,12.0,2015.5
89,3,30,6.0,18.0,12.0,2015.5


In [23]:
df3= df3[~((df3['month']==2) & (df3['day']==29))].reset_index(drop=True)
df3

,month,day,sunrise,sunset,일사,year
0,1,1,7.0,17.0,10.0,2015.5
1,1,2,7.0,17.0,10.0,2015.5
2,1,3,7.0,17.0,10.0,2015.5
3,1,4,7.0,17.0,10.0,2015.5
4,1,5,7.0,17.0,10.0,2015.5
...,...,...,...,...,...,...
85,3,27,6.0,18.0,12.0,2015.5
86,3,28,6.0,18.0,12.0,2015.5
87,3,29,6.0,18.0,12.0,2015.5
88,3,30,6.0,18.0,12.0,2015.5


In [24]:
df_date = pd.DataFrame(pd.date_range(start= '2019-01-01', end = '2019-04-01',freq='D', name='date', closed='left'))
df_date

,date
0,2019-01-01
1,2019-01-02
2,2019-01-03
3,2019-01-04
4,2019-01-05
...,...
85,2019-03-27
86,2019-03-28
87,2019-03-29
88,2019-03-30


In [25]:
test_일출일몰 = pd.concat([df_date, df3],axis=1)
test_일출일몰

,date,month,day,sunrise,sunset,일사,year
0,2019-01-01,1,1,7.0,17.0,10.0,2015.5
1,2019-01-02,1,2,7.0,17.0,10.0,2015.5
2,2019-01-03,1,3,7.0,17.0,10.0,2015.5
3,2019-01-04,1,4,7.0,17.0,10.0,2015.5
4,2019-01-05,1,5,7.0,17.0,10.0,2015.5
...,...,...,...,...,...,...,...
85,2019-03-27,3,27,6.0,18.0,12.0,2015.5
86,2019-03-28,3,28,6.0,18.0,12.0,2015.5
87,2019-03-29,3,29,6.0,18.0,12.0,2015.5
88,2019-03-30,3,30,6.0,18.0,12.0,2015.5


In [26]:
test_일출일몰 = test_일출일몰[['date', 'sunrise','sunset','일사']]
test_일출일몰

,date,sunrise,sunset,일사
0,2019-01-01,7.0,17.0,10.0
1,2019-01-02,7.0,17.0,10.0
2,2019-01-03,7.0,17.0,10.0
3,2019-01-04,7.0,17.0,10.0
4,2019-01-05,7.0,17.0,10.0
...,...,...,...,...
85,2019-03-27,6.0,18.0,12.0
86,2019-03-28,6.0,18.0,12.0
87,2019-03-29,6.0,18.0,12.0
88,2019-03-30,6.0,18.0,12.0


In [27]:
test_일출일몰.to_csv('./make_data/0C_test_일출일몰.csv', index=False)